In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 7
height = 7
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

minimum_number_of_tents = 7 # enter 0 to disable this minimum constraint

puzzle.solving_timeout = 30

puzzle.naming = {
    None: ' ',
    'n': ' ',
    'e': '_',
}

puzzle.domain_program += """
    value(e).
    value(n).
    game_value(b).
    counted_value(t;w;o).
    game_value(S) :- counted_value(S).
    value(S) :- game_value(S).

    num(0..board_width-1).
    num(0..board_height-1).
    value(N) :- num(N).
    
    guessed_number(num_offices,1).
    1 { guessed_number(num_toilets,1..2) } 1.
    
    main_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height, Col < board_width.
    bottom_counting_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row = board_height, Col < board_width.
    right_counting_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height, Col = board_width.
    counting_cell(C) :- bottom_counting_cell(C).
    counting_cell(C) :- right_counting_cell(C).
    empty_cell(C) :- cell(C),
        not main_cell(C),
        not counting_cell(C).
    
    outside_cell(c(R,C)) :- main_cell(c(R,C)), C = 1.
    outside_cell(c(R,C)) :- main_cell(c(R,C)), C = board_width-1.
    outside_cell(c(R,C)) :- main_cell(c(R,C)), R = 1.
    outside_cell(c(R,C)) :- main_cell(c(R,C)), R = board_height-1.
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : main_cell(C), solution(C,V) }.
    puzzle(C,V) :- solution(C,V), cell(C), not main_cell(C).
"""
if minimum_number_of_tents > 0:
    puzzle.puzzle_constraints_program += f"""
        :- not {minimum_number_of_tents} {{ puzzle(C,b) : main_cell(C) }}.
    """
puzzle.puzzle_constraints_program += """
    :- puzzle(C,t).
    :- puzzle(C,e).
    :- puzzle(C,w).
    puzzle(C,b) :- solution(C,b).

    #maximize { 1,C1,C2 : puzzle(C1,b), puzzle(C2,b), not adjacent_cells(C1,C2) }.
    #minimize { 10,C : counting_cell(C), solution(C,0) }.
    #minimize { 1,C : counting_cell(C), solution(C,1) }.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['adjacent_cells']
puzzle.solution_program += enc_library['ring_around_cell']

puzzle.solution_program += """
    :- main_cell(C), solution(C,V), num(V).
    :- empty_cell(C), solution(C,V), V != n.
    :- not empty_cell(C), solution(C,n).
    :- bottom_counting_cell(C), solution(C,V), not num(V).
    :- right_counting_cell(C), solution(C,V), not num(V).
    
    :- solution(C,b), main_cell(C), not puzzle(C,b).
    
    1 { match(C1,C2) : main_cell(C2), solution(C2,t), adjacent_cells(C1,C2) } 1 :-
        main_cell(C1), solution(C1,b).
    1 { match(C1,C2) : main_cell(C1), solution(C1,b), adjacent_cells(C1,C2) } 1 :-
        main_cell(C2), solution(C2,t).
    
    :- main_cell(C1), main_cell(C2), solution(C1,t), solution(C2,t),
        C1 != C2, ring_around_cell(C1,C2,1).

    :- guessed_number(num_toilets,N),
        not N { solution(C,w) : main_cell(C) } N.
        
    :- guessed_number(num_offices,N),
        not N { solution(C,o) : main_cell(C) } N.
    
    toilet_reachable_from(C) :- main_cell(C), solution(C,w).
    toilet_reachable_from(C1) :-
        main_cell(C1), main_cell(C2), adjacent_cells(C1,C2),
        toilet_reachable_from(C2), solution(C1,e).
    tent_has_toilet_coverage(C1) :-
        main_cell(C1), main_cell(C2), adjacent_cells(C1,C2),
        toilet_reachable_from(C2), solution(C1,t).
    
    tent_reachable_from(C) :- main_cell(C), solution(C,w).
    tent_reachable_from(C1) :-
        main_cell(C1), main_cell(C2), adjacent_cells(C1,C2),
        tent_reachable_from(C2), solution(C1,e).
    toilet_has_tent_coverage(C1) :-
        main_cell(C1), main_cell(C2), adjacent_cells(C1,C2),
        tent_reachable_from(C2), solution(C1,w).
    
    office_reachable_from(C) :- main_cell(C), solution(C,o).
    office_reachable_from(C1) :-
        main_cell(C1), main_cell(C2), adjacent_cells(C1,C2),
        office_reachable_from(C2), solution(C1,e).
    tent_has_office_coverage(C1) :-
        main_cell(C1), main_cell(C2), adjacent_cells(C1,C2),
        office_reachable_from(C2), solution(C1,t).
    
    :- main_cell(C), solution(C,w), not toilet_has_tent_coverage(C).
    
    :- right_counting_cell(C), C = c(Row,Col1),
        N = #count { C2 : C2 = c(Row,Col2), main_cell(C2), solution(C2,S), counted_value(S) },
        not solution(C,N).
    :- bottom_counting_cell(C), C = c(Row1,Col),
        N = #count { C2 : C2 = c(Row2,Col), main_cell(C2), solution(C2,S), counted_value(S) },
        not solution(C,N).
    
    :- solution(C,o), not outside_cell(C).
"""
puzzle.essential_solution_constraints = [
    """
    :- main_cell(C), solution(C,t), not tent_has_toilet_coverage(C).
    """,
    """
    :- main_cell(C), solution(C,t), not tent_has_office_coverage(C).
    """,
]

In [3]:
puzzle.generate(
    verbose=True,
    quality_check_criteria=["--lookahead=atom", 1, 0],
)

Adding quality check propagator..
Done grounding..
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~.
Stopped after solving timeout..
Solving time: 45.59 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

| |b| | | |b|2|
| |b| | | | |2|
| | | |b| | |1|
| | | | |b| |2|
|b| | | | | |2|
| | |b| | | |1|
|2|1|2|2|1|2| |

num_offices = 1
num_toilets = 2


In [5]:
print(puzzle.pretty_repr_solution())

|_|b|t|w|_|b|2|
|t|b|_|_|_|t|2|
|_|_|t|b|_|_|1|
|t|_|_|w|b|_|2|
|b|_|_|_|t|o|2|
|_|t|b|_|_|_|1|
|2|1|2|2|1|2| |
